In [1]:
import numpy as np
import pandas as pd

In [2]:
import re
def clean_str(string):
    string = re.sub(r"[^A-Za-z0-9(),!?\'\`]", " ", string)     
    string = re.sub(r"'s", " 's", string) 
    string = re.sub(r"'ve", " 've", string) 
    string = re.sub(r"n't", " n't", string) 
    string = re.sub(r"'re", " 're", string) 
    string = re.sub(r"'d", " 'd", string) 
    string = re.sub(r"'ll", " 'll", string) 
    string = re.sub(r",", " , ", string) 
    string = re.sub(r"!", " ! ", string) 
    string = re.sub(r"\(", " \( ", string) 
    string = re.sub(r"\)", " \) ", string) 
    string = re.sub(r"\?", " \? ", string) 
    string = re.sub(r"\s{2,}", " ", string)
    return string.strip().lower()

In [4]:
import json
from collections import defaultdict
def load_data():
    train=np.asarray(pd.read_csv('train-balanced.csv',header=None,delimiter='|'))
    comments=json.loads(open('comments.json').read())
    train_comments=[]
    vocab=defaultdict(int)
    errs=0
    for row in train:
        label=[]
        if(row[2]=='0 1'):
            label=[0,1]
        else:
            label=[1,0]
        try:
            rev=comments[row[0]]['text'].strip()
            cleaned_rev=clean_str(rev)
            words=set(cleaned_rev.split())
            for word in words:
                vocab[word]+=1
            cleaned_rev=' '.join(cleaned_rev.split()[:100])
            datum={
                "y":int(1),
                "id":row[0],
                "text":cleaned_rev,
                "author":comments[row[0]]['author'],
                "topic":comments[row[0]]['subreddit'],
                "label":label,
                "num_words":len(cleaned_rev.split()),
                "split":int(1)
            }
            train_comments.append(datum)
        except:
            errs+=1
    print("Done Loading")
    print("Errors: ",err)
    return train_comments,vocab

In [76]:
train_comments,vocab=load_data()

Done Loading
Errors:  1003


In [86]:
import io
def load_fastText(vocab):
    f=io.open('fastText/wiki-news-300d-1M.vec',encoding='latin')
    model={}
    for line in f:
        splitLine=line.split()
        word=splitLine[0]
        embedding=np.array(splitLine[1:])
        if word in vocab:
               model[word]=embedding
    return model

In [87]:
fastText=load_fastText(vocab)